In [3]:
import openpyxl 
import os
import cv2
import pytesseract
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt
from easyocr import easyocr
from pathlib import Path

pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract"


ALPHABET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
DATASET_PATH = f'dataset'
# READER = easyocr.Reader(['en'])

# shows an image on ipynb
def imshow(img):
    plt.axis('off')
    plt.grid(False)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

ModuleNotFoundError: No module named 'easyocr'

Get all testing images into an array

In [ ]:
images = []
for (id, filename) in enumerate(os.listdir(DATASET_PATH)):
    image = cv2.imread(f"{DATASET_PATH}/{filename}")
    images.append((filename, image))

Defines image cropper to center the text in the image

In [ ]:
def crop_image(img: cv2.Mat):

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (71, 71), 0)
    ret, thresh = cv2.threshold(blur,150,255,0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    big_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    x, y, w, h = cv2.boundingRect(big_contour) 
    newimg = img[y:y+h, x:x+w]

    return newimg

if __name__ == "__main__":
    img = crop_image(images[1][1])
    imshow(img)

Generate synthetic data to train neural network

In [ ]:
from trdg.generators import GeneratorFromStrings
import random

def get_random_text():
    treatment = random.choice(['C', 'D'])
    block = f"B{random.choice(['1', '2'])}"
    row = f"R{random.randint(1, 99)}"
    position = f"P{random.randint(1, 99)}"
    
    gen_length = random.randint(2, 4)
    num_count = random.choices([1, 2, 3], weights=(70, 15, 15))[0]
    num_lengths = random.choices([1, 2, 3, 4], weights=(15, 15, 15, 55), k=num_count)

    genotype = (
        "".join([ALPHABET[random.randint(0, 25)] for _ in range(gen_length)]) + 
        f"-{'-'.join([str(random.randint(10**(i-1), 10**i-1)) for i in num_lengths])}_LM"
    )

    return ','.join([treatment, block, row, position, genotype])

generator = GeneratorFromStrings(
    [get_random_text() for _ in range(1000)],
    random_blur=True,
    random_skew=True,
    blur=2,
)

for img, lbl in generator:
    img.save(f'examples/{lbl}.png')
    

Delete all the examples if needed

In [ ]:
import os
if os.name == 'nt':
    os.system('rmdir /S /Q examples') 
else:
    os.system('rm -rf examples')